In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [ ]:
import pandas as pd

In [ ]:
# change directory
cd "/content/drive/My Drive/Projects/resume_ner"

/content/drive/My Drive/Projects/resume_ner


In [ ]:
!ls

'Entity Recognition in Resumes.json'   resume_ner.ipynb   training_data.txt


In [ ]:
# load dataset
train_data = pd.read_json('Entity Recognition in Resumes.json', lines=True)

In [ ]:
# top 5 rows from dataset
train_data.head()

,content,annotation,extras
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12...",NaN
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta...",NaN
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37...",NaN
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80...",NaN
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20...",NaN


In [ ]:
train_data['content'][0]

"Abhishek Jha\nApplication Development Associate - Accenture\n\nBengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a\n\n• To work for an organization which provides me the opportunity to improve my skills\nand knowledge for my individual and company's growth in best possible ways.\n\nWilling to relocate to: Bangalore, Karnataka\n\nWORK EXPERIENCE\n\nApplication Development Associate\n\nAccenture -\n\nNovember 2017 to Present\n\nRole: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries\nfor the Bot which will be triggered based on given input. Also, Training the bot for different possible\nutterances (Both positive and negative), which will be given as\ninput by the user.\n\nEDUCATION\n\nB.E in Information science and engineering\n\nB.v.b college of engineering and technology -  Hubli, Karnataka\n\nAugust 2013 to June 2017\n\n12th in Mathematics\n\nWoodbine modern school\n\nApril 2011 to March 2013\n\n10th\n\nKendriya Vidyalaya\n\

In [ ]:
# import necessary libraries
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
clean = []

In [ ]:
# shape of dataset (rows and columns)
train_data.shape

(220, 3)

In [ ]:
!pip install spacy

### dataset from https://www.kaggle.com/dataturks/resume-entities-for-ner

In [ ]:
# chnage entities column to spacy format
# spacy format entity dictionary should be in 
# {'entities':[(start_point, end_point, label), (start_point, end_point, label), etc]}
entities = []
for index in range(220):
  anno = train_data['annotation'][index]
  entity_list = []
  for index1 in range(len(anno)):
    label = anno[index1]['label']
    if len(label) > 0:
      label = label[0]
    start = anno[index1]['points'][0]['start']
    end = anno[index1]['points'][0]['end']
    entity_list.append(tuple([start, end, label]))
    entity_dict = {'entities':entity_list}
  entities.append(entity_dict)

In [ ]:
# convert content column of dataset to list of sentences
text = train_data['content'].tolist()

In [ ]:
len(text),len(entities)

(220, 220)

In [ ]:
# initialize blank model
nlp = spacy.blank('en')
def train_model(text, entities):
  if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)

  for entity in entities:
    entity_list = entity['entities']
    for ent in range(len(entity_list)):
      label = entity_list[ent][2]
      label = str(label)
      ner.add_label(label)
  


  other_pipelines = [pipe for pipe in nlp.pipe_names if pipe!='ner']
  with nlp.disable_pipes(*other_pipelines):
    optimizer = nlp.begin_training()
    for iter in range(10):
      print(f"Start Iteration :- {iter}")
      losses = {}
      index = 0

      for index in range(220):
        try:
          nlp.update(
              [text[index]],
              [entities[index]],
              drop = 0.2,
              sgd = optimizer,
              losses = losses
          )
        except Exception as e:
          pass
      
      print(losses)

In [ ]:
train_model(text, entities)

Start Iteration :- 0
{'ner': 12815.435798814326}
Start Iteration :- 1
{'ner': 15084.808014336806}
Start Iteration :- 2
{'ner': 13408.323183883609}
Start Iteration :- 3
{'ner': 12095.71939731943}
Start Iteration :- 4
{'ner': 15019.659650223695}
Start Iteration :- 5
{'ner': 11851.329772432331}
Start Iteration :- 6
{'ner': 10523.577527572446}
Start Iteration :- 7
{'ner': 13323.196286275614}
Start Iteration :- 8
{'ner': 9137.73090569625}
Start Iteration :- 9
{'ner': 8600.095895420756}


In [ ]:
# save model
nlp.to_disk('ner_model')

In [ ]:
# load model
ner_model = spacy.load('ner_model')

In [ ]:
# take one example text from train data
example_text = train_data['content'][1]

In [ ]:
example_text

'Afreen Jamadar\nActive member of IIIT Committee in Third year\n\nSangli, Maharashtra - Email me on Indeed: indeed.com/r/Afreen-Jamadar/8baf379b705e37c6\n\nI wish to use my knowledge, skills and conceptual understanding to create excellent team\nenvironments and work consistently achieving organization objectives believes in taking initiative\nand work to excellence in my work.\n\nWORK EXPERIENCE\n\nActive member of IIIT Committee in Third year\n\nCisco Networking -  Kanpur, Uttar Pradesh\n\norganized by Techkriti IIT Kanpur and Azure Skynet.\nPERSONALLITY TRAITS:\n• Quick learning ability\n• hard working\n\nEDUCATION\n\nPG-DAC\n\nCDAC ACTS\n\n2017\n\nBachelor of Engg in Information Technology\n\nShivaji University Kolhapur -  Kolhapur, Maharashtra\n\n2016\n\nSKILLS\n\nDatabase (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT\nACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)\n\nADDITIONAL INFORMATION\n\nTECHNICAL SKILLS:\n\n• Programming 

In [ ]:
# apply trained model on exaple text to predict named entities
doc = ner_model(example_text)
for entity in doc.ents:
  print(f"{entity.label_.upper():{30}} :- {entity.text}")

EMAIL ADDRESS                  :- indeed.com/r/Afreen-Jamadar/8baf379b705e37c6
DEGREE                         :- PG
DEGREE                         :- Bachelor of Engg in Information Technology
COLLEGE NAME                   :- Shivaji University Kolhapur
SKILLS                         :- Database (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT
ACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)

ADDITIONAL INFORMATION

TECHNICAL SKILLS:

• Programming Languages: C, C++, Java, .net, php.
• Web Designing: HTML, XML
• Operating Systems: Windows […] Windows Server 2003, Linux.
• Database: MS Access, MS SQL Server 2008, Oracle 10g, MySql.

https://www.indeed.com/r/Afreen-Jamadar/8baf379b705e37c6?isid=rex-download&ikw=download-top&co=IN
